In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import data_processor
import cnn_net
import datetime
from cnn_net import CNN
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import *
import pickle
from collections import Counter
from tensorflow.contrib import learn
import data_processor as dp
from nltk import word_tokenize

## Input Data from a previously saved pickle file

In [2]:
#data = pickle.load(open("/Users/Maximus/downloads/Fulldata_wY_fullText", "rb"))
data = pickle.load(open("/root/Objects/Fulldata_wY_500", "rb"))

In [32]:
#Split data into train/dev/test sets
train_data = data.loc[data.Date < '2009-01-01', :]
dev_data = data.loc[(data.Date >= '2009-01-01') & (data.Date <= '2010-12-31'), :]
test_data = data.loc[data.Date >= '2011-01-01', :]

train_label = train_data.label
dev_label = dev_data.label
test_label = test_data.label

#Replace na with 0
train_data['Surprise'].fillna(0, inplace=True)
dev_data['Surprise'].fillna(0, inplace=True)
#train_data.fillna(0, inplace=True)
pd.options.mode.chained_assignment = None

###  Re-format Label 

In [4]:
def create_label(data_in):
#Generate new labels
  label_y = []
  for sign in data_in:
    if sign == 'DOWN':
      label_y.append([1, 0, 0])
    elif sign == 'STAY':
      label_y.append([0, 1, 0])
    else:
      label_y.append([0, 0, 1])
  
  label_y = np.asarray(label_y)
  return(label_y)

In [22]:
#This random draw is created to randomly select a smaller set of data than the entire dataset.
#Due to resource contraint, this is a more effcient way to train our model.

import random
random.seed(9)

#randomly drawn a smaller subset from training and dev set due to the large sample size
randomw_draw = random.sample(range(0, dev_data.shape[0]), 5000) 
dev_data2 = dev_data.iloc[randomw_draw,:]

randomw_draw = random.sample(range(0, train_data.shape[0]), 12000) #17000 #Total
train_data2 = train_data.iloc[randomw_draw,:]

### Embedding  Construction
##### Select the top doc_n words from input paragrahp (excluding stopwords and symbols).  Create a vocabulary, then convert into numerical indices for word embedding in TensorFlow

In [23]:
##Number of words per document to use:
doc_n = 3000

#load_features has a parser that parses the financial text so the resulting paragraph is closer to natural language
x_text = dp.load_features(train_data2.text, doc_n)

# Build Vocabulary - tranform training input words with vocab_processor
max_document_length = doc_n   #max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x_train = np.array(list(vocab_processor.fit_transform(x_text)))

print "Part 1 Done"
del x_text

#Build Vocabulary for the development set - tranform develop input words with vocab_processor
x_text = dp.load_features(dev_data2.text, doc_n)
x_dev = np.array(list(vocab_processor.transform(x_text)))

#Create the numerical feature of earning surprise for our models
surprise_train, surprise_dev = np.array(train_data2.Surprise), np.array(dev_data2.Surprise)
surprise_train = np.reshape(surprise_train, (surprise_train.shape[0], 1))
surprise_dev = np.reshape(surprise_dev, (surprise_dev.shape[0], 1))

y_train = create_label(train_data2.label)
y_dev = create_label(dev_data2.label)
print "Finished"

Part 1 Done
Finished


### Save result for future use.

In [22]:
np.save("x_train", x_train, allow_pickle=True)
np.save("x_dev", x_dev, allow_pickle=True)

In [24]:
#Delete Data to save memory
del train_data
del train_data2
del dev_data
del dev_data2
del x_text

In [30]:
#Reload Module
reload(data_processor)
reload(cnn_net)

<module 'cnn_net' from 'cnn_net.pyc'>

### We weren't able to save all the results from models with different hyperparameters.  However, below is the sample code we use to run Convolution Neural Network

### Sample CNN Implementations 

In [35]:
###Example 1 with 3000 words, large batch size 555

# Hyperparameters
######################################################
M = 124  #Embedding Size
FS = [1,2,3] #Filter Sizes
Num_F = 124 #Number of filters per filte rsize
dropout_keep_prob = 1
L = 1000
batch_size = 177
num_epoch = 100
#####################################################

#Create Dev set
dev = dp.one_iter(list(zip(x_dev, y_dev, surprise_dev))) 
for i in dev:
  x_dbatch, y_dbatch, s_dbatch = zip(*i)
         

with tf.Graph().as_default():
   
    sess = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
    with sess.as_default():
        cnn = CNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size = M,
            filter_sizes = FS,
            num_filters = Num_F,
            l2_reg_lambda = L)

        #Setup Training 
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        
        # Initialize 
        sess.run(tf.initialize_all_variables())

        def train_step(x_batch, y_batch, s_batch):
            """
            One Training Step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step,loss, accuracy = sess.run(
                [train_op, global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            if step % 100 == 0:
              print("\nTrain Checkpoint:")
              print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            
        
        def dev_step(x_batch, y_batch, s_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            step, loss, accuracy, predictions = sess.run(
                [global_step, cnn.loss, cnn.accuracy, cnn.predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
           
          
        # Generate training batches
        batches = data_processor.batch_iter(
            list(zip(x_train, y_train, surprise_train)), batch_size, num_epoch)
        
        for batch in batches:
            x_batch, y_batch, s_batch = zip(*batch)
            train_step(x_batch, y_batch, s_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            
            if current_step % 200 == 0:
                print("\nDev Set Evaluation:")
                dev_step(x_dbatch, y_dbatch, s_dbatch)


Train Checkpoint:
2016-12-18T10:57:47.849883: step 100, loss 150.289, acc 0.559322

Train Checkpoint:
2016-12-18T11:22:23.189610: step 200, loss 3.755, acc 0.333333

Dev Set Evaluation:
2016-12-18T11:24:52.496297: step 200, loss 3.63621, acc 0.3934

Train Checkpoint:
2016-12-18T11:49:43.539803: step 300, loss 1.09656, acc 0.451977

Train Checkpoint:
2016-12-18T12:14:10.384034: step 400, loss 1.09018, acc 0.38983

Dev Set Evaluation:
2016-12-18T12:16:39.098753: step 400, loss 1.09448, acc 0.3928

Train Checkpoint:
2016-12-18T12:41:17.630859: step 500, loss 1.07667, acc 0.451977

Train Checkpoint:
2016-12-18T13:05:57.252270: step 600, loss 1.09293, acc 0.423729

Dev Set Evaluation:
2016-12-18T13:08:24.188452: step 600, loss 1.09319, acc 0.3862

Train Checkpoint:
2016-12-18T13:33:05.291789: step 700, loss 1.0989, acc 0.355932

Train Checkpoint:
2016-12-18T13:57:43.233941: step 800, loss 1.08071, acc 0.38983

Dev Set Evaluation:
2016-12-18T14:00:14.117969: step 800, loss 1.10521, acc 0.40

KeyboardInterrupt: 

###  Smaller batch (74) with 2000 words per record

In [26]:
#########################
##Start Training       ##
#########################


# Hyperparameters
######################################################
M = 124  #Embedding Size
FS = [1,2,3] #Filter Sizes
Num_F = 124 #Number of filters per filte rsize
dropout_keep_prob = 1
L = 10000
batch_size = 74
num_epoch = 100
#####################################################

#Create Dev set
dev = dp.one_iter(list(zip(x_dev, y_dev, surprise_dev))) 
for i in dev:
  x_dbatch, y_dbatch, s_dbatch = zip(*i)
         

with tf.Graph().as_default():
   
    sess = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
    with sess.as_default():
        cnn = CNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size = M,
            filter_sizes = FS,
            num_filters = Num_F,
            l2_reg_lambda = L)

        #Setup Training 
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Initialize 
        sess.run(tf.initialize_all_variables())

        def train_step(x_batch, y_batch, s_batch):
            """
            One Training Step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step,loss, accuracy = sess.run(
                [train_op, global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            if step % 100 == 0:
              print("\nTrain Checkpoint:")
              print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            
        
        def dev_step(x_batch, y_batch, s_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            step, loss, accuracy = sess.run(
                [global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))     
           
          
        # Generate batches
        batches = data_processor.batch_iter(
            list(zip(x_train, y_train, surprise_train)), batch_size, num_epoch)
        
        for batch in batches:
            x_batch, y_batch, s_batch = zip(*batch)
            train_step(x_batch, y_batch, s_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            
            if current_step % 200 == 0:
                print("\nDev Set Evaluation:")
                dev_step(x_dbatch, y_dbatch, s_dbatch)


Train Checkpoint:
2016-12-17T10:59:38.691790: step 100, loss 1741.64, acc 0.310811

Train Checkpoint:
2016-12-17T11:10:55.210979: step 200, loss 34.1061, acc 0.310811

Dev Set Evaluation:
2016-12-17T11:13:23.590202: step 200, loss 32.8919, acc 0.2546

Train Checkpoint:
2016-12-17T11:24:54.318806: step 300, loss 1.25294, acc 0.391892

Train Checkpoint:
2016-12-17T11:36:09.566092: step 400, loss 1.09923, acc 0.364865

Dev Set Evaluation:
2016-12-17T11:38:40.972046: step 400, loss 1.09446, acc 0.4058

Train Checkpoint:
2016-12-17T11:50:02.222860: step 500, loss 1.09985, acc 0.351351

Train Checkpoint:
2016-12-17T12:01:25.880083: step 600, loss 1.10384, acc 0.391892

Dev Set Evaluation:
2016-12-17T12:04:02.816054: step 600, loss 1.09474, acc 0.3996

Train Checkpoint:
2016-12-17T12:15:16.512116: step 700, loss 1.0982, acc 0.405405

Train Checkpoint:
2016-12-17T12:26:42.247399: step 800, loss 1.10486, acc 0.324324

Dev Set Evaluation:
2016-12-17T12:29:10.937594: step 800, loss 1.0948, acc 0

KeyboardInterrupt: 

In [ ]:
###Example 2 with 3000 words, large batch size 215

# Hyperparameters
######################################################
M = 124  #Embedding Size
FS = [2,3,4] #Filter Sizes
Num_F = 124 #Number of filters per filte rsize
dropout_keep_prob = 1
L = 10000
batch_size = 215
num_epoch = 100
#####################################################

#Create Dev set
dev = dp.one_iter(list(zip(x_dev, y_dev, surprise_dev))) 
for i in dev:
  x_dbatch, y_dbatch, s_dbatch = zip(*i)
         

with tf.Graph().as_default():
   
    sess = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
    with sess.as_default():
        cnn = CNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size = M,
            filter_sizes = FS,
            num_filters = Num_F,
            l2_reg_lambda = L)

        #Setup Training 
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        
        # Initialize 
        sess.run(tf.initialize_all_variables())

        def train_step(x_batch, y_batch, s_batch):
            """
            One Training Step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step,loss, accuracy = sess.run(
                [train_op, global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            if step % 100 == 0:
              print("\nTrain Checkpoint:")
              print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            
        
        def dev_step(x_batch, y_batch, s_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            step, loss, accuracy = sess.run(
                [global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))     
           
          
        # Generate training batches
        batches = data_processor.batch_iter(
            list(zip(x_train, y_train, surprise_train)), batch_size, num_epoch)
        
        for batch in batches:
            x_batch, y_batch, s_batch = zip(*batch)
            train_step(x_batch, y_batch, s_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            
            if current_step % 200 == 0:
                print("\nDev Set Evaluation:")
                dev_step(x_dbatch, y_dbatch, s_dbatch)

In [22]:
#####
##Start Training
#####

# Hyperparameters
######################################################
M = 124  #Embedding Size
FS = [1,2,3] #Filter Sizes
Num_F = 124 #Number of filters per filte rsize
dropout_keep_prob = 1
L = 10000
batch_size = 75
num_epoch = 100
#####################################################

with tf.Graph().as_default():
   
    sess = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
    with sess.as_default():
        cnn = CNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size = M,
            filter_sizes = FS,
            num_filters = Num_F,
            l2_reg_lambda = L)

        #Setup Training 
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        
        
        # Initialize 
        sess.run(tf.initialize_all_variables())

        def train_step(x_batch, y_batch, s_batch):
            """
            One Training Step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step,loss, accuracy = sess.run(
                [train_op, global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            if step % 100 == 0:
              print("\nTrain Checkpoint:")
              print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            
        
        def dev_step(x_batch, y_batch, s_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            step, loss, accuracy = sess.run(
                [global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))     
           
          
        # Generate batches
        batches = data_processor.batch_iter(
            list(zip(x_train, y_train, surprise_train)), batch_size, num_epoch)
        
        for batch in batches:
            x_batch, y_batch, s_batch = zip(*batch)
            train_step(x_batch, y_batch, s_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            if current_step % 100 == 0:
                print("\nDev Set Evaluation:")
                dev_step(x_dev, y_dev, surprise_dev)


Train Checkpoint:
2016-12-14T07:37:48.960365: step 100, loss 1801.02, acc 0.506667

Dev Set Evaluation:
2016-12-14T07:38:24.624259: step 100, loss 1741.53, acc 0.496

Train Checkpoint:
2016-12-14T07:42:01.253003: step 200, loss 36.1494, acc 0.48

Dev Set Evaluation:
2016-12-14T07:42:36.143203: step 200, loss 34.5943, acc 0.44

Train Checkpoint:
2016-12-14T07:46:20.647712: step 300, loss 1.28033, acc 0.426667

Dev Set Evaluation:
2016-12-14T07:46:58.458832: step 300, loss 1.25946, acc 0.411333

Train Checkpoint:
2016-12-14T07:51:01.637704: step 400, loss 1.09757, acc 0.386667

Dev Set Evaluation:
2016-12-14T07:51:39.850304: step 400, loss 1.0953, acc 0.402667

Train Checkpoint:
2016-12-14T07:55:44.956532: step 500, loss 1.09913, acc 0.4

Dev Set Evaluation:
2016-12-14T07:56:24.322188: step 500, loss 1.09484, acc 0.410333

Train Checkpoint:
2016-12-14T08:00:27.721672: step 600, loss 1.09869, acc 0.386667

Dev Set Evaluation:
2016-12-14T08:01:07.457427: step 600, loss 1.09655, acc 0.4163

KeyboardInterrupt: 

In [9]:
#####
##Start Training
#####

# Hyperparameters
######################################################
M = 124  #Embedding Size
FS = [3,4,5] #Filter Sizes
Num_F = 124 #Number of filters per filte rsize
dropout_keep_prob = 1
L = 100
batch_size = 75
num_epoch = 100
#####################################################

with tf.Graph().as_default():
   
    sess = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
    with sess.as_default():
        cnn = CNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size = M,
            filter_sizes = FS,
            num_filters = Num_F,
            l2_reg_lambda = L)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)


        # Output directory for models and summaries
        #timestamp = str(int(time.time()))
        #out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        #print("Writing to {}\n".format(out_dir))

        
        # Summaries for loss and accuracy
        #loss_summary = tf.scalar_summary("loss", cnn.loss)
        #acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)

        # Train Summaries
        #train_summary_op = tf.merge_summary([loss_summary, acc_summary, grad_summaries_merged])
        #train_summary_dir = os.path.join(out_dir, "summaries", "train")
        #train_summary_writer = tf.train.SummaryWriter(train_summary_dir, sess.graph)


        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        #checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        #checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        #if not os.path.exists(checkpoint_dir):
        #    os.makedirs(checkpoint_dir)
        #saver = tf.train.Saver(tf.all_variables())

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.initialize_all_variables())

        def train_step(x_batch, y_batch, s_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step,loss, accuracy = sess.run(
                [train_op, global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            if step % 50 == 0:
              print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

        
        def dev_step(x_batch, y_batch, s_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.input_suprise: s_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            step, loss, accuracy = sess.run(
                [global_step, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            if step % 50 == 0:
              print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))     
           
        # Generate batches
        batches = data_processor.batch_iter(
            list(zip(x_train, y_train, surprise_train)), batch_size, num_epoch)
        # Training loop. For each batch...
        
        for batch in batches:
            x_batch, y_batch, s_batch = zip(*batch)
            train_step(x_batch, y_batch, s_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            if current_step % 100 == 0:
                print("\nDev set Evaluation:")
                dev_step(x_dev, y_dev, surprise_dev)

2016-12-13T06:51:34.165871: step 50, loss 80.0191, acc 0.4
2016-12-13T06:53:39.802057: step 100, loss 17.1919, acc 0.346667

Dev set Evaluation:
2016-12-13T06:54:25.884365: step 100, loss 16.6229, acc 0.372
2016-12-13T06:56:30.860248: step 150, loss 3.56904, acc 0.453333
2016-12-13T06:58:36.826978: step 200, loss 1.34105, acc 0.453333

Dev set Evaluation:
2016-12-13T06:59:22.740437: step 200, loss 1.44876, acc 0.396333
2016-12-13T07:01:28.282701: step 250, loss 1.10076, acc 0.4
2016-12-13T07:03:33.206529: step 300, loss 1.04698, acc 0.466667

Dev set Evaluation:
2016-12-13T07:04:19.456236: step 300, loss 1.13621, acc 0.438333
2016-12-13T07:06:25.403664: step 350, loss 1.09456, acc 0.32
2016-12-13T07:08:30.895494: step 400, loss 1.04159, acc 0.493333

Dev set Evaluation:
2016-12-13T07:09:17.100089: step 400, loss 1.1571, acc 0.391333
2016-12-13T07:11:21.145860: step 450, loss 1.08969, acc 0.36
2016-12-13T07:13:27.814229: step 500, loss 1.10441, acc 0.36

Dev set Evaluation:
2016-12-13T0

KeyboardInterrupt: 

In [ ]:
test = dp.one_iter(list(zip(x_test, y_test, surprise_test))) 
for i in test:
  x_btest, y_btest, s_btest = zip(*i)

x_text = dp.load_features(test.text, doc_n)
x_test = np.array(list(vocab_processor.transform(x_text)))

#Create the numerical feature of earning surprise for our models
surprise_test = np.array(test.Surprise, (surprise_test.shape[0], 1))

y_test = create_label(test.label)